# Use Linkup with agents.json
This notebook demonstrates how to use agents.json to interact with the Stripe API. This simple example shows loading the agents.json file and using it to execute Stripe tasks.


In [1]:
# %pip install agentsjson
# %pip install openai

### Setup API keys and references to `agents.json` files

In [2]:
LINKUP_API_KEY="your-linkup-api-key"
OPENAI_API_KEY="your-openai-api-key" 
agents_json_url = "https://gist.githubusercontent.com/yagnyaPatel/41278eb9056006a8313d4e5313cf9735/raw/6c66385ec143ff4681110583de4be8ba25f7e858/linkup.agents.json"

In [3]:
import requests

# Fetch the agents.json file from the provided URL
response = requests.get(agents_json_url)
if response.status_code == 200:
    print(response.text)
else:
    raise Exception(f"Failed to fetch agents.json file: {response.status_code}")


{
  "agentsJson": "0.1.0",
  "info": {
    "title": "Link Up API Integration Agents",
    "version": "0.1.0",
    "description": "This agents.json specification integrates with the Link Up API, which allows you to retrieve web content. It provides a flow for searching the web with different output types including sourced answers, search results, or structured data formats."
  },
  "sources": [
    {
      "id": "linkup",
      "path": "https://gist.githubusercontent.com/yagnyaPatel/670dd9fec7d101b76f92a2c4f7acfba5/raw/5fd960dfdebd35016dc285bc52aa7051576b1f66/linkup.openapi.yaml",
      "description": "The Link Up OpenAPI specification covering web search and content retrieval functionality."
    }
  ],
  "overrides": [],
  "flows": [
    {
      "id": "linkup_search_flow",
      "title": "Search the Web",
      "description": "Retrieves web content based on a natural language query. Can return results in different formats: sourced answers with citations, search results, or custom struc

### Load the agents.json file

In [4]:
from agentsjson.core.models import Flow
from agentsjson.core.models.bundle import Bundle
from agentsjson.core import load_agents_json

# load the agents.json file
data: Bundle = load_agents_json(agents_json_url)
agentsjson = data.agentsJson

### Define the Agent

In [5]:
query = "What time is it in New York?"

In [6]:
from agentsjson import ToolFormat, get_tool_prompt

# Format the flows data for the prompt
flows_context = get_tool_prompt(agentsjson)

# Create the system prompt
system_prompt = f"""You are an AI assistant that helps users interact with the Linkup Web Search API.
You have access to the following API flows:

{flows_context}

Analyze the user's request and use the appropriate API flows to accomplish the task.
You must give your arguments for the tool call as Structued Outputs JSON with keys `parameters` and `requestBody`"""

### Configure Auth

In [7]:
from agentsjson.core.models.auth import AuthType, ApiKeyAuthConfig
auth = ApiKeyAuthConfig(type=AuthType.API_KEY, key_value=LINKUP_API_KEY)

### Run the agent

In [12]:
from openai import OpenAI
from agentsjson import execute, get_tools
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ],
    tools=get_tools(agentsjson, format=ToolFormat.OPENAI),
    temperature=0,
    tool_choice="required"
)

response = execute(agentsjson, response, format=ToolFormat.OPENAI, auth=auth)

response



{'linkup_search_flow': {'success': LinkupSourcedAnswer(answer='The current time in New York is 6:37 AM on February 27, 2025.', sources=[LinkupSource(name='Current time in New York, United States.', url='https://clocks.world/time/united-states/new-york/', snippet='The time in New York is 5 hours behind UTC and the same as in Ashburn. The current timezone in New York is Eastern Standard Time (EST). New York is currently observing Standard Time (DST) also known as winter time.\nThe time in New York is 5 hours behind UTC and the same as in Ashburn. The current timezone in New York is Eastern Standard Time (EST). New York is currently observing Standard Time (DST) also known as winter time.\nDaylight Saving Time in New York starts at March 9, 2025. New York uses Daylight Saving Time.'), LinkupSource(name='Current time in New York', url='https://en.true-time.com/us-ny/', snippet='Current time in New York New York — exact time with seconds online. New York — transition to winter and summer ti